In [1]:
import osmnx as ox
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image
Image.MAX_IMAGE_PIXELS = 933120000

import warnings
warnings.filterwarnings('ignore', '.*do not.*', )
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

%matplotlib inline
ox.__version__

C:\Users\wei.909\AppData\Local\Temp\2\ipykernel_11572\2409756490.py:10: UserWarning: DelftStack
  warnings.warn('DelftStack')


'1.1.2'

In [2]:
def create_square_from_osm(addr, bbox, dist, tags={'building':True}, dpi=3000):
    #bbox = ox.utils_geo.bbox_from_point(point, dist=dist)
    gdf_building = ox.geometries.geometries_from_point(addr, tags=tags, dist=dist)

    if not gdf_building.empty:
        fig, ax = ox.plot_footprints(gdf_building, ax=None, bbox=bbox, filepath=f'./sat_label.png', bgcolor='#000000', dpi=dpi,
                                     save=True, show=False, close=True)

In [3]:
# set up GIS map bounday, center point and distance
# Please refer to MapAnalysis for target map width and height
north, south = 40.00050846915017, 39.99829017146142
east, west = -83.0139618955573, -83.01911201346185
satmap_center = (39.99939933, -83.01653695)
dist = 300

create_square_from_osm(addr=satmap_center, bbox=(north, south, east, west), dist=dist)

C:\Users\wei.909\AppData\Local\Continuum\anaconda3\envs\GISgenerator2021\lib\site-packages\geopandas\plotting.py:74: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if mpl >= LooseVersion("3.4") or (mpl > LooseVersion("3.3.2") and "+" in mpl):
C:\Users\wei.909\AppData\Local\Continuum\anaconda3\envs\GISgenerator2021\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [4]:
# Combine Satellite map with GIS building mask for visualization
sat_img = cv2.imread('./sat.png')
sat_label = cv2.imread('./sat_label.png')

h_b, w_b = sat_img.shape[:2]
sat_label = cv2.resize(sat_label, (w_b, h_b))
save_img = 0.5*sat_img+0.5*sat_label
cv2.imwrite('sat_mask.png', save_img)

True

In [5]:
# the three channels
white = np.array([0, 0, 0])

sat_mask = np.ones((sat_label.shape[:2]), dtype=np.int32)
sat_mask[(sat_label==white).all(axis=2)] = 0

im = Image.fromarray(sat_mask)
im.save("mask.png")